In [9]:
import math

class LZW:
    def __init__(self, text):
        self.text = text
        self.encoded_string = self.compress()
        self.bits_before = self.compute_bits_before()
        self.bits_after = self.compute_bits_after()
        self.compression_ratio = self.compute_cr()
        self.probabilities = self.compute_probabilities()
        self.entropy = self.compute_entropy()
        self.efficiency = self.compute_efficiency()

    def compress(self):
        if not self.text:
            return []

        dictionary = {chr(i): i for i in range(128)}
        current_code = 128
        compressed = []
        current_string = ""

        for char in self.text:
            if current_string + char in dictionary:
                current_string += char
            else:
                compressed.append(dictionary[current_string])
                dictionary[current_string + char] = current_code
                current_code += 1
                current_string = char

        if current_string:
            compressed.append(dictionary[current_string])

        return compressed

    def compute_bits_before(self):
        bits_before = 0
        for char in self.text:
            if char.isdigit():
                bits_before += 1
            elif char.isalpha():
                bits_before += 8
        return bits_before

    def compute_bits_after(self):
        encoding = math.ceil(math.log2(max(self.encoded_string) + 1))
        bits_after = (len(self.encoded_string) ) * encoding
        return bits_after

    def compute_cr(self):
        original_size = self.bits_before
        compressed_size = self.bits_after
        return original_size / compressed_size if compressed_size > 0 else 0

    def compute_probabilities(self):
        freq_map = {}
        total_symbols = 0
        for symbol in self.text:
            freq_map[symbol] = freq_map.get(symbol, 0) + 1
            total_symbols += 1
        char_prob = {symbol: freq / total_symbols for symbol, freq in freq_map.items()}
        return char_prob

    def compute_entropy(self):
        entropy = 0
        total_symbols = len(self.text)
        char_prob = self.probabilities
        for freq in char_prob.values():
            entropy -= freq * math.log2(freq)
        return entropy

    def compute_efficiency(self):
        original_size = self.bits_before
        compressed_size = self.bits_after
        return ((original_size - compressed_size) / original_size) * 100 if original_size > 0 else 0

    def get_results(self):
        return {
            "encoded_string": self.encoded_string,
            "bits_before": self.bits_before,
            "bits_after": self.bits_after,
            "compression ratio (%)": round(self.compression_ratio * 100, 1),
            "probabilities": self.probabilities,
            "entropy": round(self.entropy, 3),
            "efficiency": round(self.efficiency, 1),
        }
    
    @staticmethod
    def lzw_decompress(compressed):
        if not compressed:
            return ""

        dictionary = {i: chr(i) for i in range(128)}  # ASCII characters from 0 to 127
        current_code = 128  # Start new codes from 128
        decompressed = []

        previous = chr(compressed.pop(0))
        decompressed.append(previous)

        for code in compressed:
            if code in dictionary:
                entry = dictionary[code]
            elif code == current_code:
                entry = previous + previous[0]
            else:
                raise ValueError("Bad compressed sequence")

            decompressed.append(entry)

            if current_code < 256:  # Max code value in 0-128 range
                dictionary[current_code] = previous + entry[0]
                current_code += 1

            previous = entry

        return ''.join(decompressed)

text = "wabbawabbadrewsr fyht g"
lzw = LZW(text)
print(lzw.get_results())
decompressed_text = LZW.lzw_decompress(lzw.encoded_string)
print("Decompressed text:", decompressed_text)


{'encoded_string': [119, 97, 98, 98, 97, 128, 130, 97, 100, 114, 101, 119, 115, 114, 32, 102, 121, 104, 116, 32, 103], 'bits_before': 168, 'bits_after': 168, 'compression ratio (%)': 100.0, 'probabilities': {'w': 0.13043478260869565, 'a': 0.17391304347826086, 'b': 0.17391304347826086, 'd': 0.043478260869565216, 'r': 0.08695652173913043, 'e': 0.043478260869565216, 's': 0.043478260869565216, ' ': 0.08695652173913043, 'f': 0.043478260869565216, 'y': 0.043478260869565216, 'h': 0.043478260869565216, 't': 0.043478260869565216, 'g': 0.043478260869565216}, 'entropy': 3.447, 'efficiency': 0.0}
Decompressed text: wabbawabbadrewsr fyht g
wabbawabbadrewsr fyht g
